First download the hymenoptera datasat

In [3]:
from pathlib import Path
import zipfile
from loguru import logger

In [4]:
def get_hymenoptera():
    datadir = Path.home() / ".cache/mads_datasets/hymenoptera_data"
    url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
    if not datadir.exists():
        logger.info(f"Creating directory {datadir}")
        datadir.mkdir(parents=True)

        response = requests.get(url)
        zip_file_path = datadir / "hymenoptera_data.zip"
        with open(zip_file_path, "wb") as f:
            f.write(response.content)

        logger.info(f"Extracting {zip_file_path}")
        with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
            zip_ref.extractall(datadir)
        zip_file_path.unlink()
    else:
        logger.info(f"Directory {datadir} already exists, skipping download.")
    return datadir / "hymenoptera_data"

datadir = get_hymenoptera()

2025-10-11 15:11:29.912 | INFO     | __main__:get_hymenoptera:18 - Directory /Users/wouter/.cache/mads_datasets/hymenoptera_data already exists, skipping download.


Then print the output to check what is inside.

In [6]:
from pathlib import Path
import os

datadir = get_hymenoptera()

# print het pad zelf
print("Dataset directory:", datadir)

# print de inhoud van de hoofdfolder
print("Inhoud van de dataset folder:")
for item in datadir.iterdir():
    print(" -", item)

# eventueel ook dieper kijken in train/val
train_dir = datadir / "train"
val_dir = datadir / "val"

print("\nTrain classes:")
for cls in train_dir.iterdir():
    print(f" - {cls.name}: {len(list(cls.iterdir()))} images")

print("\nValidation classes:")
for cls in val_dir.iterdir():
    print(f" - {cls.name}: {len(list(cls.iterdir()))} images")

2025-10-11 15:25:46.840 | INFO     | __main__:get_hymenoptera:18 - Directory /Users/wouter/.cache/mads_datasets/hymenoptera_data already exists, skipping download.


Dataset directory: /Users/wouter/.cache/mads_datasets/hymenoptera_data/hymenoptera_data
Inhoud van de dataset folder:
 - /Users/wouter/.cache/mads_datasets/hymenoptera_data/hymenoptera_data/train
 - /Users/wouter/.cache/mads_datasets/hymenoptera_data/hymenoptera_data/val

Train classes:
 - bees: 121 images
 - ants: 124 images

Validation classes:
 - bees: 83 images
 - ants: 70 images


Hypothesis:
A convolutional neural network with multiple conv2d layers, batch normalization, and dropout will achieve higher classification accuracy on the hymenoptera dataset than a simple linear model, because the spatial structure in the images (wings, body shape, antennae) can be better captured by convolutional layers.